In [1]:
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from scipy.stats.mstats import ttest_ind
from scipy.stats import f
import numpy as np
import statsmodels.formula.api as smf

In [2]:
from download import download
url = "https://www.data.gouv.fr/fr/datasets/r/6fadff46-9efd-4c53-942a-54aca783c30c"
path_target = "datasets/donnees-hospitalieres-nouveaux-covid19-2020-05-29-19h00.csv"
download(url, path_target, replace=False)

Replace is False and data exists, so doing nothing. Use replace==True to re-download the data.


'datasets/donnees-hospitalieres-nouveaux-covid19-2020-05-29-19h00.csv'

In [3]:
url="https://www.insee.fr/fr/statistiques/fichier/1893198/estim-pop-dep-sexe-gca-1975-2020.xls"
path_target="datasets/PopFrance2020"
download(url, path_target, replace=False)

Replace is False and data exists, so doing nothing. Use replace==True to re-download the data.


'datasets/PopFrance2020'

In [4]:
df_Covidfr = pd.read_csv('datasets/donnees-hospitalieres-nouveaux-covid19-2020-05-29-19h00.csv',sep = ';')

In [5]:
df_popFrance2020=pd.read_excel('datasets/PopFrance2020',sheet_name=1,skiprows=4,skipfooter=4, dtype={'Unnamed: 0':str}).rename(columns={'Unnamed: 0':'dep'})


In [6]:
ListeDep=np.sort(df_Covidfr['dep'].unique())

In [7]:
df_ParDepartement=pd.DataFrame(0,index=ListeDep,columns=['hosp','rea','dc','pop','ratio1','ratio2','ratio3','ratio4'])

for dep in ListeDep :
   df_ParDepartement.loc[dep,'hosp']=(df_Covidfr[(df_Covidfr['dep']==dep)]['incid_hosp']).sum()
   df_ParDepartement.loc[dep,'rea']=(df_Covidfr[(df_Covidfr['dep']==dep)]['incid_rea']).sum()
   df_ParDepartement.loc[dep,'dc']=(df_Covidfr[(df_Covidfr['dep']==dep)]['incid_dc']).sum()
   df_ParDepartement.loc[dep,'pop']=(df_popFrance2020[(df_popFrance2020['dep']==dep)]['Total']).sum()

In [8]:
df_ParDepartement.dtypes

hosp      int64
rea       int64
dc        int64
pop       int64
ratio1    int64
ratio2    int64
ratio3    int64
ratio4    int64
dtype: object

In [9]:
df_ParDepartement.loc['13']

hosp         4300
rea           677
dc            531
pop       2034469
ratio1          0
ratio2          0
ratio3          0
ratio4          0
Name: 13, dtype: int64

In [10]:
#ratio1 nombre de morts par millions d'habitants
df_ParDepartement['ratio1']=df_ParDepartement['dc']/df_ParDepartement['pop']*1000000
#ratio2 Nombre de dc par milliers d"hospitalisés
df_ParDepartement['ratio2']=df_ParDepartement['dc']/df_ParDepartement['hosp']*1000
#ratio3 Nombre de dc par centaines de cas de réa 
df_ParDepartement['ratio3']=df_ParDepartement['dc']/df_ParDepartement['rea']*100
#ratio4 Nombre de réa par centaine d'horpitalisés 
df_ParDepartement['ratio4']=df_ParDepartement['rea']/df_ParDepartement['hosp']*100

In [30]:
#calcul de la moyenne 
df_ParDepartement.mean()

hosp        1068.712871
rea          173.306931
dc           185.663366
pop       663997.059406
ratio1       235.555831
ratio2       156.657672
ratio3       111.305689
ratio4        16.060802
dtype: float64

In [31]:
#On compare avec les résultats dans le 13
df_ParDepartement.loc['13']

hosp      4.300000e+03
rea       6.770000e+02
dc        5.310000e+02
pop       2.034469e+06
ratio1    2.610018e+02
ratio2    1.234884e+02
ratio3    7.843427e+01
ratio4    1.574419e+01
Name: 13, dtype: float64

In [11]:
#on convertit en nombre entier pour la visibilité
df_ParDepartementEntier=df_ParDepartement.astype('int')


In [22]:
df_ParDepartementEntier['ratio1'].rank(method='min').loc['13']

70.0

In [23]:
df_ParDepartementEntier['ratio2'].rank(method='min').loc['13']


23.0

In [24]:
df_ParDepartementEntier['ratio3'].rank(method='min').loc['13']

25.0

In [25]:
df_ParDepartementEntier['ratio4'].rank(method='min').loc['13']

49.0

In [21]:
df_ParDepartementEntier.sort_values(by='ratio1').head(70)

,hosp,rea,dc,pop,ratio1,ratio2,ratio3,ratio4
974,142,19,1,859959,1,7,5,13
973,98,5,1,290691,3,10,20,5
48,24,2,1,76286,13,41,50,8
82,55,20,4,262618,15,72,20,36
09,46,10,3,152398,19,65,30,21
...,...,...,...,...,...,...,...,...
39,389,57,61,257849,236,156,107,14
26,711,134,132,520560,253,185,98,18
89,437,48,85,332096,255,194,177,10
18,304,35,76,296404,256,250,217,11


In [27]:
df_ParDepartementEntier.sort_values(by='ratio2').head(30)

,hosp,rea,dc,pop,ratio1,ratio2,ratio3,ratio4
974,142,19,1,859959,1,7,5,13
973,98,5,1,290691,3,10,20,5
48,24,2,1,76286,13,41,50,8
976,307,39,19,279471,67,61,48,12
2B,155,13,10,182258,54,64,76,8
09,46,10,3,152398,19,65,30,21
82,55,20,4,262618,15,72,20,36
24,178,14,13,408393,31,73,92,7
04,155,11,12,165197,72,77,109,7
66,370,90,35,479000,73,94,38,24


In [28]:
df_ParDepartementEntier.sort_values(by='ratio3').head(30)

,hosp,rea,dc,pop,ratio1,ratio2,ratio3,ratio4
974,142,19,1,859959,1,7,5,13
973,98,5,1,290691,3,10,20,5
82,55,20,4,262618,15,72,20,36
09,46,10,3,152398,19,65,30,21
31,722,210,69,1400935,49,95,32,29
23,108,33,11,116270,94,101,33,30
66,370,90,35,479000,73,94,38,24
971,123,46,21,376879,55,170,45,37
976,307,39,19,279471,67,61,48,12
45,839,202,98,682890,143,116,48,24


In [29]:
df_ParDepartementEntier.sort_values(by='ratio4').head(50)

,hosp,rea,dc,pop,ratio1,ratio2,ratio3,ratio4
973,98,5,1,290691,3,10,20,5
10,943,61,137,309907,442,145,224,6
04,155,11,12,165197,72,77,109,7
65,197,14,26,226839,114,131,185,7
46,198,15,29,173166,167,146,193,7
24,178,14,13,408393,31,73,92,7
43,152,12,15,226901,66,98,125,7
02,1439,123,261,526050,496,181,212,8
90,720,58,177,140145,1262,245,305,8
07,652,54,109,326875,333,167,201,8
